In [ ]:
import torch
from tqdm import tqdm

# Image-related utilities
from torchvision.io import decode_image, read_image
from torchvision.transforms import ToTensor
from torchvision import transforms
from PIL import Image

# Import models
from torchvision.models import vgg19, VGG19_Weights

# Dataset
from torchvision.datasets import Imagenette, ImageFolder

# Plotting utility
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Read imagenette data into data loader
#Process image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to VGG19 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Same normalization as ImageNet
])
imagenette_data = ImageFolder(root='/home/yi/Downloads/imagenette2/val', transform=transform)
data_loader = torch.utils.data.DataLoader(imagenette_data, batch_size=1, shuffle=False)


In [ ]:
# For pre-trained model
# Map number to class name
label_to_class = {
    0: "English springer",
    1: "French horn",
    2: "cassette player",
    3: "chain saw",
    4: "church",
    5: "garbage truck",
    6: "gas pump",
    7: "golf ball",
    8: "parachute",
    9: "tench"
}

label_to_id = {
    0: 217,
    1: 566,
    2: 482, 
    3: 491,
    4: 497,
    5: 569,
    6: 571, 
    7: 574,
    8: 701, 
    9: 0
}

# For retrained model
# Manually rewire predicted class_id to class
pred_to_class = {
    1: 'English springer',
    2: 'cassette player',
    3: 'chain saw',
    4: 'church',
    5: 'French horn',
    6: 'garbage truck',
    7: 'gas pump',
    8: 'golf ball',
    9: 'parachute',
    0: 'tench'
}

label_rewire = {
    0: 1,
    1: 5, 
    2: 2,
    3: 3,
    4: 4,
    5: 6,
    6: 7,
    7: 8,
    8: 9,
    9: 0
}

In [ ]:
# Pre-trained model
weights = VGG19_Weights.DEFAULT
model = vgg19(weights=weights).to(device)
model.eval()

# Retrained model
'''
PATH = 'vgg19_imagenette.pth'
model = vgg19()
model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10)
model.load_state_dict(torch.load(PATH, weights_only=True))
model = model.to(device)
model.eval()
'''

In [ ]:
# Evaluation (output to csv)
def evaluate(data_loader, PATH):
    img_path = []
    true_label = []
    predicted_label = []
    confidence = []
    true_confidence = []
    correctness = []
    
    image_paths = [data_loader.dataset.samples[i][0] for i in range(len(data_loader.dataset))]
    
    with torch.no_grad():
        for idx, (img, label) in enumerate(tqdm(data_loader, desc="Evaluating", unit="images")):
            # Retrieve image
            img = img.to(device)
            label = label.to(device)
            
            # Prediction
            prediction = model(img).squeeze(0).softmax(0)
            class_id = prediction.argmax().item()
            score = prediction[class_id].item()
            true_score = prediction[label_rewire[label.item()]].item()
            
            # Correct?
            #category_name = weights.meta["categories"][class_id]
            
            # Extract class names from the dataset
            id_to_class = {v: k for k, v in imagenette_data.class_to_idx.items()}  # Reverse mapping
            # Get predicted category name
            category_name = pred_to_class[class_id]
            # Check if prediction is correct
            correct = 'y' if category_name == id_to_class[label.item()] else 'n'
            
            # Log
            img_path.append(image_paths[idx])
            true_label.append(id_to_class[label.item()])
            predicted_label.append(category_name)
            confidence.append(score)
            true_confidence.append(true_score)
            correctness.append(correct)
    
    # Dictionary of logs
    dict = {'image': img_path, 'true label': true_label, 'predicted label': predicted_label,
            'confidence score': confidence, 'true confidence score': true_confidence, 'correct': correctness}
    df = pd.DataFrame(dict)
    df.to_csv(PATH)

In [ ]:
# Print evaluation accuracy
print('Evaluation accuracy: ', round(len(df[df['correct'] == 'y']) / len(df), 4))